Все импорты должны выполняться, особенно если вы запускаете это внутри докера

In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'
style.use('dark_background')
from IPython.display import clear_output
from copy import copy, deepcopy
from tqdm import tqdm_notebook as tm
from tqdm import trange
from glob import glob
import os
import sys
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import pandas.io.sql as psql

Вспомогательные функции для запроса данных из базы

In [3]:
def get_conn():
    import psycopg2 as pg
    return pg.connect("dbname=vk_data user=vk_parser password=WegEWWXfedLf2YbS host=13.84.188.132")

def run_sql(query):
    import pandas.io.sql as psql
    res = psql.read_sql(query, get_conn())
    try:
        return res.set_index('uid')
    except:
        import warnings
        warnings.warn("[run_sql] UID wasn't selected, default index used")
        return res

def get_uid():
    return run_sql("SELECT uid FROM users WHERE data_loaded")

In [4]:
# Всего у нас вот столько таблиц

run_sql("""SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'""")

,table_name
0,groups
1,likes
2,posts
3,users_friends
4,users_groups
5,users


In [5]:
# Посмотреть на все "колонки" в таблице можно вот так

run_sql("Select * FROM users LIMIT 0")

,first_name,last_name,sex,nickname,domain,bdate,city,country,photo_50,photo_100,photo_200_orig,can_write_private_message,status,data_loaded,mobile_phone,home_phone,group_list_loaded,comments_loaded,is_leader
uid,,,,,,,,,,,,,,,,,,,


In [6]:
# Так можно посмотреть на всех Ань

df = run_sql("SELECT uid FROM users WHERE first_name = 'Аня'")
df.head()

""
uid
191922123
6639568
2454912
2068667
6136199


In [7]:
# А так - сделать поиск по тексту комментария

df = run_sql("SELECT from_id, text FROM posts  WHERE (text <> '') AND (text LIKE '% да ладно тебе%')")
df.head()

,from_id,text
0,36596923,ой да ладно тебе=)\nя думаю что ти думаешь что...
1,82380,"Ой, да ладно тебе;)) Стиснительный тут нашелся..."
2,626037,"ой да ладно тебе)) можно подумать, ты плохой))..."
3,979411,"Ой, да ладно тебе))))"
4,391773,Ваньк) да ладно тебе) справишься!!! уж я то то...


In [1]:
def getPostsByUid(uid):
    sqlQuery = "SELECT from_id, text FROM posts  WHERE (text <> '') AND (from_id = " + str(uid) + ")"
    return run_sql(sqlQuery).text

In [15]:
getPostsByUid(15759198)

0                                       хм...в смысле?)
1     не я все так удаляю))оставляю ток картинки и м...
2                    аххах ну это все для дебя детка)))
3     ой не повериш как я искала искала хд))а потом ...
4                           аххаха вот сейчас)))))хд)))
5                            пашка!!!а я тебя нашла))хд
6                                  урааа пасипки зай)))
7     хд да ладно тебе)))))))я же шучу насчет кулисо...
8     хд ну пашечка ну солнышко..ну малыш..ну кися))...
9                 аххах а ты меня возешь с собой???))))
10    пасип зайк))обязательно буду)))хахах)))*лыбить...
11    ооххохо прикольно))я та тоже не плохо это а ко...
12    оооо пашка ну ты отрастил патлы))))мужичек))))...
Name: text, dtype: object

In [11]:
# Допустим теперь для каждого юзера у нас появились предсказания, запишем их в сабмишн файл 

ids = get_uid()
leaders = np.random.choice(list(ids.index), size=100)


submission = pd.DataFrame({'is_leader':np.zeros(ids.shape[0])}, index=ids.index)
submission.loc[leaders] = 1

submission.to_csv('../data/submission/random_submission.csv')